# Disease Prediction and Medical Recommendation System

# Import Libraries and load dataset

In [1]:
import pandas as pd
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import numpy as np
import pickle

dataset = pd.read_csv('kaggle_dataset/Training.csv')

# Splitting into train and test dataset

In [2]:
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']

le = LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

In [3]:
np.random.seed(100)
noise_indices = np.random.choice(len(X_test), size=int(len(X_test) * 0.25), replace=False)
for idx in noise_indices:
    flip_count = np.random.randint(10, 16)
    flip_indices = np.random.choice(X_test.shape[1], size=flip_count, replace=False)
    for feature_idx in flip_indices:
        X_test.iloc[idx, feature_idx] = 1 - X_test.iloc[idx, feature_idx] 

# Training the prediction models

In [4]:
prediction_models = {
    'SVC': SVC(kernel='linear', C=0.1),  
    'RandomForest': RandomForestClassifier(
        n_estimators=50,                 
        max_depth=10,                    
        min_samples_split=5,            
        max_features='sqrt',            
        random_state=42
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=50,                
        learning_rate=0.08,             
        max_depth=8,                    
        min_samples_split=4,            
        random_state=42
    ),
}

for name_of_model, model in prediction_models.items():
    
    model.fit(X_train, y_train)                                                

    test_predictions = model.predict(X_test)                                    

    model_accuracy = accuracy_score(y_test, test_predictions)                    
    print(f"{name_of_model} Accuracy: {model_accuracy:.4f} \n")

    cm = confusion_matrix(y_test, test_predictions)
    print(f"{name_of_model} Confusion Matrix: \n")
    print(np.array2string(cm, separator=', '))

SVC Accuracy: 0.9966 

SVC Confusion Matrix: 

[[36,  0,  0, ...,  0,  0,  0],
 [ 0, 36,  0, ...,  0,  0,  0],
 [ 0,  0, 36, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 36,  0,  0],
 [ 0,  0,  0, ...,  0, 36,  0],
 [ 0,  0,  0, ...,  0,  0, 36]]
RandomForest Accuracy: 0.8848 

RandomForest Confusion Matrix: 

[[32,  0,  0, ...,  0,  0,  0],
 [ 0, 25,  0, ...,  0,  1,  0],
 [ 1,  0, 30, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 28,  0,  0],
 [ 0,  0,  0, ...,  0, 36,  0],
 [ 1,  0,  0, ...,  0,  1, 31]]
GradientBoosting Accuracy: 0.8083 

GradientBoosting Confusion Matrix: 

[[24,  0,  0, ...,  0,  0,  0],
 [ 0, 26,  0, ...,  0,  1,  0],
 [ 0,  0, 29, ...,  0,  1,  0],
 ...,
 [ 0,  0,  0, ..., 28,  0,  0],
 [ 0,  0,  0, ...,  0, 30,  0],
 [ 0,  0,  0, ...,  1,  0, 31]]


### Printing the precision, recall and f1-score using the classification report function.

In [5]:
print("\n", classification_report(y_test, test_predictions))


               precision    recall  f1-score   support

           0       1.00      0.67      0.80        36
           1       1.00      0.72      0.84        36
           2       0.97      0.81      0.88        36
           3       0.97      0.81      0.88        36
           4       0.91      0.81      0.85        36
           5       1.00      0.78      0.88        36
           6       1.00      0.75      0.86        36
           7       0.96      0.72      0.83        36
           8       1.00      0.69      0.82        36
           9       1.00      0.75      0.86        36
          10       1.00      0.75      0.86        36
          11       0.97      0.78      0.86        36
          12       1.00      0.81      0.89        36
          13       1.00      0.81      0.89        36
          14       0.74      0.81      0.77        36
          15       0.83      0.83      0.83        36
          16       0.62      0.89      0.73        36
          17       0.60  

# Selecting Random Forest model

In [6]:
Rf = RandomForestClassifier(n_estimators=100, random_state=42)
Rf.fit(X_train, y_train)
ypred = Rf.predict(X_test)
print(f"RandomForest Final Accuracy: {accuracy_score(y_test, ypred):.4f}")

RandomForest Final Accuracy: 0.9370


# Saving and loading the pickle file

In [7]:
import pickle
pickle.dump(Rf, open('model/RandomForest.pkl', 'wb'))
Rf = pickle.load(open('model/RandomForest.pkl', 'rb'))

# Testing the predictions

In [8]:
print("Disease Predicted :", le.inverse_transform(Rf.predict(X_test.iloc[[0]])))
print("Actual Disease :", le.inverse_transform([y_test[0]]))

Disease Predicted : ['Hyperthyroidism']
Actual Disease : ['Hyperthyroidism']


In [9]:
print("Disease Predicted :", le.inverse_transform(Rf.predict(X_test.iloc[[100]])))
print("Actual Disease :", le.inverse_transform([y_test[100]]))

Disease Predicted : ['Peptic ulcer diseae']
Actual Disease : ['Peptic ulcer diseae']


# Loading the database for Prediction 

In [10]:
symptoms = pd.read_csv("kaggle_dataset/symptoms_df.csv")
precautions = pd.read_csv("kaggle_dataset/precautions_df.csv")
workout = pd.read_csv("kaggle_dataset/workout_df.csv")
description = pd.read_csv("kaggle_dataset/description.csv")
medications = pd.read_csv('kaggle_dataset/medications.csv')
diets = pd.read_csv("kaggle_dataset/diets.csv")

###  Making disctionary of symptoms and diseases
### Building a information function to extract the description, precaution, medication, diet and workout details from the dataset

In [11]:
symptoms_list = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}

symptoms_list_processed = {symptom.replace('_', ' ').lower(): value for symptom, value in symptoms_list.items()}

def information(predicted_dis):
    disease_description = description[description['Disease'] == predicted_dis]['Description']
    disease_description = " ".join([w for w in disease_description])

    disease_precautions = precautions[precautions['Disease'] == predicted_dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    disease_precautions = [col for col in disease_precautions.values]

    disease_medications = medications[medications['Disease'] == predicted_dis]['Medication']
    disease_medications = [med for med in disease_medications.values]

    disease_diet = diets[diets['Disease'] == predicted_dis]['Diet']
    disease_diet = [die for die in disease_diet.values]

    disease_workout = workout[workout['disease'] == predicted_dis]['workout']
    
    return disease_description, disease_precautions, disease_medications, disease_diet, disease_workout


### Prediction Model function

In [12]:
def predicted_value(patient_symptoms):
    i_vector = np.zeros(len(symptoms_list_processed))
    for i in patient_symptoms:
        i_vector[symptoms_list_processed[i]] = 1
    
    return le.inverse_transform([Rf.predict([i_vector])[0]])[0]

In [13]:
from fuzzywuzzy import process

def correct_spelling(symptom):
    matched_symptom, score = process.extractOne(symptom, list(symptoms_list_processed.keys()))
    
    if score >= 80:
        return matched_symptom
    else:
        return None


In [14]:
symptoms = input("Describe your symptoms: ")
patient_symptoms = [s.strip() for s in symptoms.split(',')] 

corrected_symptoms = []
for symptom in patient_symptoms:
    corrected_symptom = correct_spelling(symptom.lower())
    if corrected_symptom:
        corrected_symptoms.append(corrected_symptom)
    else:
        print(f"Symptom '{symptom}' not found in the database.")

if corrected_symptoms:
    predicted_disease = predicted_value(corrected_symptoms)
    disease_description, disease_precautions, disease_medications, disease_diet, disease_workout = information(predicted_disease)

    print("\nPredicted Disease:", predicted_disease)
    print("\nDescription:")
    print(disease_description)
    print("\nPrecautions:")
    for idx, precaution in enumerate(disease_precautions[0], start=1):
        print(f"{idx}. {precaution}")
    print("\nMedications:")
    for idx, medication in enumerate(disease_medications, start=1):
        print(f"{idx}. {medication}")
    print("\nWorkout:")
    for idx, workout in enumerate(disease_workout, start=1):
        print(f"{idx}. {workout}")
    print("\nDiets:")
    for idx, diet in enumerate(disease_diet, start=1):
        print(f"{idx}. {diet}")




Predicted Disease: Heart attack

Description:
Heart attack is a sudden and severe reduction in blood flow to the heart muscle.

Precautions:
1. call ambulance
2. chew or swallow asprin
3. keep calm
4. nan

Medications:
1. ['Compression stockings', 'Exercise', 'Elevating the legs', 'Sclerotherapy', 'Laser treatments']

Workout:
1. Follow a heart-healthy diet
2. Limit sodium intake
3. Include fiber-rich foods
4. Consume healthy fats
5. Include lean proteins
6. Limit sugary foods and beverages
7. Stay hydrated
8. Consult a healthcare professional
9. Follow medical recommendations
10. Engage in regular exercise

Diets:
1. ['Heart-Healthy Diet', 'Low-sodium foods', 'Fruits and vegetables', 'Whole grains', 'Lean proteins']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
